# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [37]:
import pandas as pd
import numpy as np

# Load the CSV with custom missing value indicators
missing_values = ['-', 'missing', 'null', 'NaN', '']
df = pd.read_csv('production_data.csv', na_values=missing_values)

# Clean and transform each column as per the specification

# batch_id: should not have missing values
if df['batch_id'].isnull().any():
    raise ValueError("Missing values found in 'batch_id' which is not allowed.")

# production_date
df['production_date'] = pd.to_datetime(df['production_date'], errors='coerce')

# raw_material_supplier
df['raw_material_supplier'] = df['raw_material_supplier'].map({1: 'national_supplier', 2: 'international_supplier'})
df['raw_material_supplier'] = df['raw_material_supplier'].fillna('national_supplier')

# pigment_type
df['pigment_type'] = df['pigment_type'].str.lower().replace({'type_a': 'type_a', 'type_b': 'type_b', 'type_c': 'type_c'})
valid_pigments = ['type_a', 'type_b', 'type_c']
df['pigment_type'] = df['pigment_type'].where(df['pigment_type'].isin(valid_pigments), other='other')

# pigment_quantity
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df['pigment_quantity'] = df['pigment_quantity'].clip(lower=1, upper=100)
df['pigment_quantity'] = df['pigment_quantity'].fillna(df['pigment_quantity'].median())

# mixing_time
df['mixing_time'] = pd.to_numeric(df['mixing_time'], errors='coerce')
df['mixing_time'] = df['mixing_time'].fillna(round(df['mixing_time'].mean(), 2))

# mixing_speed
df['mixing_speed'] = df['mixing_speed'].astype(str).str.strip()
valid_speeds = ['Low', 'Medium', 'High']
df['mixing_speed'] = df['mixing_speed'].where(df['mixing_speed'].isin(valid_speeds), other='Not Specified')

# product_quality_score
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')
df['product_quality_score'] = df['product_quality_score'].fillna(round(df['product_quality_score'].mean(), 2))

# Final clean DataFrame
clean_data = df.copy()
clean_data.head(10)


,batch_id,production_date,raw_material_supplier,pigment_type,pigment_quantity,mixing_time,mixing_speed,product_quality_score
0,1,2024-06-25,national_supplier,type_a,42.822881,22.56,High,7.165102
1,2,2023-11-23,international_supplier,type_b,42.873479,44.97,High,6.849126
2,3,2024-02-18,international_supplier,type_b,33.251206,52.62,High,5.661209
3,4,2023-11-11,national_supplier,type_a,52.532817,24.96,High,6.991735
4,5,2024-04-11,international_supplier,type_c,36.205108,25.37,High,7.095043
5,6,2024-02-14,international_supplier,type_b,29.991903,45.96,Medium,6.136382
6,7,2023-08-25,international_supplier,type_a,35.941439,32.13,High,5.735791
7,8,2024-04-26,international_supplier,type_a,40.497203,7.75,Medium,5.510766
8,9,2024-07-06,international_supplier,type_a,36.015111,35.28,Low,4.959952
9,10,2023-11-21,national_supplier,type_a,42.893169,18.61,High,9.876712


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [38]:
aggregated_data = df.groupby('raw_material_supplier').agg({
    'product_quality_score': 'mean',
    'pigment_quantity': 'mean'
}).rename(columns={
    'product_quality_score': 'avg_product_quality_score',
    'pigment_quantity': 'avg_pigment_quantity'
}).reset_index()

aggregated_data = aggregated_data.round(2)
aggregated_data


,raw_material_supplier,avg_product_quality_score,avg_pigment_quantity
0,international_supplier,5.97,34.91
1,national_supplier,8.02,44.73


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [39]:
# Load original CSV
df = pd.read_csv('production_data.csv')

# step 1: Filter data according to business conditions
filtered_df = df[
    (df['raw_material_supplier'] == 2) &
    (df['pigment_quantity'] > 35)
]

# step 2 : Dealing with missing values

# production_date column
filtered_df['production_date'] = pd.to_datetime(filtered_df['production_date'], errors='coerce')

# raw_material_supplier column
filtered_df['raw_material_supplier'] = filtered_df['raw_material_supplier'].astype('category')
filtered_df['raw_material_supplier'] = filtered_df['raw_material_supplier'].fillna('national_supplier')

# pigment_type column
filtered_df['pigment_type'] = filtered_df['pigment_type'].astype('category')
filtered_df['pigment_type'] = filtered_df['pigment_type'].replace('Type_C','type_c')
filtered_df['pigment_type'] = filtered_df['pigment_type'].fillna('other')

# pigment_quantity
filtered_df['pigment_quantity'] = filtered_df['pigment_quantity'].fillna(filtered_df['pigment_quantity'].median())

# mixing_time column
filtered_df['mixing_time'] = filtered_df['mixing_time'].fillna(round(filtered_df['mixing_time'].mean(), 2))

# mixing_speed column
filtered_df['mixing_speed'] = filtered_df['mixing_speed'].astype('category')
filtered_df['mixing_speed'] = filtered_df['mixing_speed'].replace('-','Not Specified')

# product_quality_score column
filtered_df['product_quality_score'] = filtered_df['product_quality_score'].fillna(round(filtered_df['product_quality_score'].mean(), 2))

# step 3: Compute averages from filtered data
avg_quantity = round(filtered_df['pigment_quantity'].mean(), 2)
avg_score = round(filtered_df['product_quality_score'].mean(), 2)

# step 4: Build result DataFrame with required structure
pigment_data = pd.DataFrame([{
    'raw_material_supplier': 2,
    'pigment_quantity': avg_quantity,
    'avg_product_quality_score': avg_score
}])

# Display
pigment_data

,raw_material_supplier,pigment_quantity,avg_product_quality_score
0,2,39.01,5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [40]:
#Calculate the mean and standard deviation for pigment_quantity
pigment_quantity_mean = round(clean_data['pigment_quantity'].mean(), 2)
pigment_quantity_sd = round(clean_data['pigment_quantity'].std(), 2)

#Calculate the mean and standard deviation for product_quality_score
product_quality_score_mean = round(clean_data['product_quality_score'].mean(), 2)
product_quality_score_sd = round(clean_data['product_quality_score'].std(), 2)

#Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score
corr_coef = round(clean_data['pigment_quantity'].corr(clean_data['product_quality_score']), 2)

#product_quality DataFrame
product_quality = pd.DataFrame([{
    'product_quality_score_mean': product_quality_score_mean,
    'product_quality_score_sd': product_quality_score_sd,
    'pigment_quantity_mean': pigment_quantity_mean,
    'pigment_quantity_sd': pigment_quantity_sd,
    'corr_coef': corr_coef
}])

product_quality

,product_quality_score_mean,product_quality_score_sd,pigment_quantity_mean,pigment_quantity_sd,corr_coef
0,6.68,1.39,38.35,6.83,0.49
